# Test suite for Project 1


In [71]:
from proj1_helpers import *
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

## Data loading

* `../data/train.csv`
* `../data/test.csv`


In [14]:
train_path = '../data/train.csv'
test_path  = '../data/test.csv'

In [15]:
# Long, run only if necessary
train_data = load_csv_data(train_path)

In [16]:
# Long, run only if necessary
test_data = load_csv_data(test_path)

In [17]:
# Save the data as pickles for more efficient reloading
# Only run once to generate the pickle !
# pickle.dump(test_data, open( 'test.p', 'wb' ))
# pickle.dump(train_data, open( 'train.p', 'wb' ))

In [19]:
# loading the pickle back 
# train_data = pickle.load(open( 'train.p', 'rb' ))
# test_data = pickle.load(open( 'test.p', 'rb' ))

## Exploratory

* Describe / Discover the data
* Describe / Characterise the features

In [49]:
## Data dimensions
print("There are {} samples.".format(len(train_data[0])))

There are 250000 samples.


The training data is defined as a set $D = \{ (y_i, \mathbf{x}_i, w_i) \}$ with :
* $y_i \in \{+1,-1\}$ is the label (signal = `+1` or noise = `-1`)
* $\mathbf{x}_i \in \!R^d$ is a $d$-dimensional feature vector
* $w_i \in \!R^+$ is a non-negative weight

Note that $\sum_{i\in\mathcal{S}} w_i = N_s$ and $\sum_{i\in\mathcal{B}} w_i = N_b$ which are the *expected total number of signal and background events (resp.)*. This gives an estimate of how many events we should expect to classify for each class.

In [50]:
# The feature vector shape is 
train_data[1].shape

(250000, 30)

In [52]:
# Which means we have d features :
d = train_data[1].shape[1]
d

30

In [53]:
labels  = train_data[0]
x_train = train_data[1]
weights = train_data[2]

Information about the features :

* Variables are floating point unless specified otherwise.
* All azimuthal φ angles are in radian in the [−π, +π[ range.
* Energy, mass, momentum are all in GeV
* All other variables are unit less
* **Undefined values are `-999.0`**

There are `primitive` (prefixed with `PRI`) values, directly measured from the collision, and `derived` (prefixed with `DER`) values which were computed from the primitive values.

In [134]:
PRI_features = x_train[:,1:15]
DER_features = x_train[:,15:]
print(len(DER_features), len(PRI_features))

250000 250000


## Data processing

* Split test / train
* Clean useless features
* [Scale / process data](https://machinelearningmastery.com/discover-feature-engineering-how-to-engineer-features-and-how-to-get-good-at-it/) (e.g. scaling)

How much data is perfectly clean ?

In [118]:
clean_data = [x for x in x_train if not -999. in x]
len(clean_data)/len(x_train)*100 # percent

27.245599999999996

How much data is clean in the primitive or derived classes ?

In [121]:
PRI_clean_data = [x for x in PRI_features if not -999. in x]
len(PRI_clean_data)/len(PRI_features)*100 # percent

29.0172

In [123]:
DER_clean_data = [x for x in DER_features if not -999. in x]
len(DER_clean_data)/len(DER_features)*100 # percent

29.0172

The percentage is the same ! This means there was no treatment error, using only `PRI` or only `DER` data won't make the data cleaner (but it might help reduce the complexity of the model ;) )

Are there classes that contain a high probability of dirty data ?

In [128]:
per_feature_stats = []
for i in range(d):
    sz = len(x_train[:,i])
    clean_ft = [x for x in x_train[:,i] if (x != -999.)] # c pa bo ... :'(
    per_feature_stats.append(len(clean_ft) / sz * 100)

In [129]:
per_feature_stats

[84.75439999999999,
 100.0,
 100.0,
 100.0,
 29.0172,
 29.0172,
 29.0172,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 29.0172,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 100.0,
 60.0348,
 60.0348,
 60.0348,
 29.0172,
 29.0172,
 29.0172,
 100.0]

This gives a pretty good indication of the features that should be pruned from the data ! Notably :
* `x_train[:, 4: 7]` (which are resp. `DER_deltaeta_jet_jet`, `DER_mass_jet_jet`, `DER_prodeta_jet_jet`)
* `x_train[:,12]` (which is `DER_lep_eta_centrality`)
* `x_train[:,26:29]` (which is the `PRI_jet_subleading_{pt,eta,phi}`)

We can also consider removing `x_train[:,23:26]` (`PRI_jet_leading_{pt,eta,phi}`) which only has ~60% clean data ...

In [170]:
def build_idx(ranges):
    clean_idx = np.asarray([list(range(i,j)) for i,j in ranges])
    return [item for sublist in clean_idx for item in sublist]

In [171]:
# Only 100% clean data
ranges = [(1,4), (7,12), (13,26), (29,30)]
keep_idx = build_idx(ranges)
x_train_full_clean = x_train[:,keep_idx]

In [172]:
# With feature 1
ranges = [(0,4), (7,12), (13,26), (29,30)]
keep_idx = build_idx(ranges)
x_train_partial_clean = x_train[:,keep_idx]

## Training

* **Train** the model with the different learning algorithms
    * `least_squares`
    * `least_squares_GD`
    * `least_squares_SGD`
    * `ridge_regression`
    * `logistic_regression`
    * `reg_logistic_regression`

## Testing

* **Test** the model with the weights computed from the different learning algorithms to find the best one

## Cross-validate

* the hyperparameters for each algorithm
    * `least_squares` 
    * `least_squares_GD` -> `gamma`
    * `least_squares_SGD` -> `gamma`, `batch_size`
    * `ridge_regression` -> `lambda_`
    * `logistic_regression` -> `gamma`
    * `reg_logistic_regression` -> `lambda_`, `gamma`
* Also, if we use other features (e.g. polynomial), we should CV those as well

## Results

* Plot the train / test accuracies for the best set of algorithm + parameters